Statistical Learning 1  
Ramiro Wong  
23000952

# Proyecto 1 - Fase 2

In [68]:
import pandas as pd
import numpy as np
import xgboost
import lightgbm

In [69]:
# modelos
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

# metricas
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

# gestion de train-test
from sklearn.model_selection import train_test_split

# transformaciones
from sklearn.preprocessing import MinMaxScaler

# hiperparámetros
from sklearn.model_selection import GridSearchCV

# tiempos
from datetime import datetime

In [11]:
completeData = pd.read_csv('output_ing_car.csv')
completeData.head()

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,1,0,1,0,0.812271,0,0,1,0,0,0,0,0,0,0,1,2,24.808856,5.466529,0
1,0,0,0,0,8.707660,1,1,1,0,0,0,0,0,0,1,0,3,45.352363,22.785731,0
2,0,0,0,0,1.417776,1,1,1,0,0,0,0,0,0,0,1,3,43.046130,9.022086,1
3,0,0,0,0,10.127377,0,0,1,0,0,0,0,0,0,1,0,0,34.363919,22.609166,0
4,1,0,0,0,1.417776,1,1,2,0,0,0,0,0,0,0,1,2,55.479087,10.173582,1


### Selección de target y features

In [22]:
X = completeData.drop(columns=["Churn", "gender", "MultipleLines", "StreamingTV", "StreamingMovies"]) # quitamos variable target y las que en el EDA definimos que no aportan valor
y = completeData["Churn"]

### Split entre entrenamiento y prueba

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2025, shuffle=True)

### Creación y aplicación del escalado de datos

In [24]:
scaler = MinMaxScaler()

scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Construcción de Modelos

### 4a - Naive-Bayes

Entrenamiento

In [82]:
hyper_parameters = {
    "priors":[None, [0.2, 0.8], [0.4, 0.6], [0.5, 0.5], [0.6, 0.4]],
    "var_smoothing":[1e-9, 1e-8, 1e-7, 1e-6, 1e-5]
}

nb_classifier = GaussianNB()

start_time = datetime.now() # hora de inicio entrenamiento

# configuración de organización de hiper-parámetros
nb_hyp_opt = GridSearchCV(estimator=nb_classifier, param_grid=hyper_parameters, cv=10, scoring="roc_auc")

# entrenamiento de modelo
nb_hyp_opt.fit(X_train_scaled, y_train)

end_time = datetime.now() # hora de final entrenamiento
training_duration = end_time - start_time # tiempo total entrenamiento

print("Hora inicio training: ", start_time.strftime("%Y-%m-%d %H:%M:%S"))
print("Hora final training: ", end_time.strftime("%Y-%m-%d %H:%M:%S"))
print("Duración training: ", training_duration)

Hora inicio training:  2024-06-30 20:04:42
Hora final training:  2024-06-30 20:04:49
Duración training:  0:00:06.522450


Ranking de modelos

In [86]:
pd.DataFrame(nb_hyp_opt.cv_results_).sort_values(by="rank_test_score",ascending=True)[["rank_test_score", "params", "mean_test_score", "std_test_score"]]

,rank_test_score,params,mean_test_score,std_test_score
0,1,"{'priors': None, 'var_smoothing': 1e-09}",0.829083,0.014991
22,1,"{'priors': [0.6, 0.4], 'var_smoothing': 1e-07}",0.829083,0.014991
21,1,"{'priors': [0.6, 0.4], 'var_smoothing': 1e-08}",0.829083,0.014991
20,1,"{'priors': [0.6, 0.4], 'var_smoothing': 1e-09}",0.829083,0.014991
18,1,"{'priors': [0.5, 0.5], 'var_smoothing': 1e-06}",0.829083,0.014991
17,1,"{'priors': [0.5, 0.5], 'var_smoothing': 1e-07}",0.829083,0.014991
16,1,"{'priors': [0.5, 0.5], 'var_smoothing': 1e-08}",0.829083,0.014991
15,1,"{'priors': [0.5, 0.5], 'var_smoothing': 1e-09}",0.829083,0.014991
13,1,"{'priors': [0.4, 0.6], 'var_smoothing': 1e-06}",0.829083,0.014991
23,1,"{'priors': [0.6, 0.4], 'var_smoothing': 1e-06}",0.829083,0.014991


Registro del modelo

In [88]:
nb_v1 = GaussianNB(priors=None, var_smoothing=1e-09)

nb_v1.fit(X_train_scaled, y_train) 

nb_v1_predicts = nb_v1.predict(X_test_scaled)

roc_auc = roc_auc_score(nb_v1_predicts, y_test)
print("Roc_Auc_score", roc_auc)

Roc_Auc_score 0.7067425231682554


Modelo final

In [90]:
nb_modelo_final = GaussianNB(priors=None, var_smoothing=1e-09)

X_scaled = scaler.transform(X)
nb_modelo_final.fit(X_scaled, y) 

GaussianNB()

### 4b - LDA

Entrenamiento

In [70]:
hyper_parameters = {
    "solver":["svd", "lsqr", "eigen"],
    "shrinkage":[None, "auto", 0.5],
    "priors":[None, [0.2, 0.8], [0.4, 0.6], [0.5, 0.5], [0.6, 0.4]],
}

LDA = LinearDiscriminantAnalysis()

start_time = datetime.now() # hora de inicio entrenamiento

# configuración de organización de hiper-parámetros
LDA_hyp_opt = GridSearchCV(estimator=LDA, param_grid=hyper_parameters, cv=10, scoring="roc_auc")

# entrenamiento de modelo
LDA_hyp_opt.fit(X_train_scaled, y_train)

end_time = datetime.now() # hora de final entrenamiento
training_duration = end_time - start_time # tiempo total entrenamiento

print("Hora inicio training: ", start_time.strftime("%Y-%m-%d %H:%M:%S"))
print("Hora final training: ", end_time.strftime("%Y-%m-%d %H:%M:%S"))
print("Duración training: ", training_duration)

Hora inicio training:  2024-06-29 23:14:12
Hora final training:  2024-06-29 23:14:24
Duración training:  0:00:11.649079


c:\Users\USUARIO\anaconda\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
150 fits failed out of a total of 450.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
50 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\USUARIO\anaconda\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\USUARIO\anaconda\Lib\site-packages\sklearn\base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\USUARIO\anaconda\Lib\site-packages\sklearn\discriminant_analysis.py", line 647, in fit
    self._

Ranking de modelos

In [78]:
pd.DataFrame(LDA_hyp_opt.cv_results_).sort_values(by="rank_test_score",ascending=True)[["rank_test_score", "params", "mean_test_score", "std_test_score"]]

,rank_test_score,params,mean_test_score,std_test_score
28,1,"{'priors': [0.5, 0.5], 'shrinkage': None, 'solver': 'lsqr'}",0.842511,0.016387
23,2,"{'priors': [0.4, 0.6], 'shrinkage': 'auto', 'solver': 'eigen'}",0.842394,0.016618
22,3,"{'priors': [0.4, 0.6], 'shrinkage': 'auto', 'solver': 'lsqr'}",0.842393,0.016618
19,4,"{'priors': [0.4, 0.6], 'shrinkage': None, 'solver': 'lsqr'}",0.842387,0.016581
32,5,"{'priors': [0.5, 0.5], 'shrinkage': 'auto', 'solver': 'eigen'}",0.842321,0.016482
31,6,"{'priors': [0.5, 0.5], 'shrinkage': 'auto', 'solver': 'lsqr'}",0.842320,0.016482
37,7,"{'priors': [0.6, 0.4], 'shrinkage': None, 'solver': 'lsqr'}",0.841905,0.016477
41,8,"{'priors': [0.6, 0.4], 'shrinkage': 'auto', 'solver': 'eigen'}",0.841524,0.016757
40,8,"{'priors': [0.6, 0.4], 'shrinkage': 'auto', 'solver': 'lsqr'}",0.841524,0.016757
1,10,"{'priors': None, 'shrinkage': None, 'solver': 'lsqr'}",0.840128,0.016640


Registro del modelo

In [73]:
LDA_v1 = LinearDiscriminantAnalysis(priors=[0.5, 0.5], solver='lsqr',shrinkage=None)

LDA_v1.fit(X_train_scaled, y_train) 

LDA_v1_predicts = LDA_v1.predict(X_test_scaled)

roc_auc = roc_auc_score(LDA_v1_predicts, y_test)
print("Roc_Auc_score", roc_auc)

Roc_Auc_score 0.7055541815517083


Modelo final

In [74]:
LDA_modelo_final = LinearDiscriminantAnalysis(priors=[0.5, 0.5], solver='lsqr',shrinkage=None)

X_scaled = scaler.transform(X)
LDA_modelo_final.fit(X_scaled, y) 

LinearDiscriminantAnalysis(priors=[0.5, 0.5], solver='lsqr')

### 4c - Regresión Logística

Entrenamiento

In [76]:
hyper_parameters = {
    "penalty":[ "l1", "l2"],
    "C":[0, 0.25, 0.5, 0.75, 1.0],
    "solver":["lbfgs", "liblinear", "newton-cg", "newton-cholesky", "sag"],
    "max_iter":[100, 150, 200, 250, 300]
}

logit = LogisticRegression()

start_time = datetime.now() # hora de inicio entrenamiento

# configuración de organización de hiper-parámetros
logit_hyp_opt = GridSearchCV(estimator=logit, param_grid=hyper_parameters, cv=10, scoring="roc_auc")

# entrenamiento de modelo
logit_hyp_opt.fit(X_train_scaled, y_train)

end_time = datetime.now() # hora de final entrenamiento
training_duration = end_time - start_time # tiempo total entrenamiento

print("Hora inicio training: ", start_time.strftime("%Y-%m-%d %H:%M:%S"))
print("Hora final training: ", end_time.strftime("%Y-%m-%d %H:%M:%S"))
print("Duración training: ", training_duration)

Hora inicio training:  2024-06-30 19:45:48
Hora final training:  2024-06-30 19:47:02
Duración training:  0:01:14.276983


c:\Users\USUARIO\anaconda\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
1300 fits failed out of a total of 2500.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
500 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\USUARIO\anaconda\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\USUARIO\anaconda\Lib\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "c:\Users\USUARIO\anaconda\Lib\site-packages\sklearn\base.py", line 666, in _validate_params
    validate_parameter_constraints(
  File "c:\Users\USUARIO\anaconda\Lib\site

In [56]:
logit_hyp_opt.best_estimator_

LogisticRegression(C=0.2, max_iter=150, penalty=None, solver='sag')

Ranking de modelos

In [83]:
pd.DataFrame(logit_hyp_opt.cv_results_).sort_values(by="rank_test_score",ascending=True)[["rank_test_score", "params", "mean_test_score", "std_test_score"]]

,rank_test_score,params,mean_test_score,std_test_score
181,1,"{'C': 0.75, 'max_iter': 250, 'penalty': 'l1', 'solver': 'liblinear'}",0.842266,0.017009
191,2,"{'C': 0.75, 'max_iter': 300, 'penalty': 'l1', 'solver': 'liblinear'}",0.842264,0.017020
171,3,"{'C': 0.75, 'max_iter': 200, 'penalty': 'l1', 'solver': 'liblinear'}",0.842262,0.017014
161,4,"{'C': 0.75, 'max_iter': 150, 'penalty': 'l1', 'solver': 'liblinear'}",0.842260,0.017016
111,5,"{'C': 0.5, 'max_iter': 150, 'penalty': 'l1', 'solver': 'liblinear'}",0.842256,0.016963
...,...,...,...,...
84,121,"{'C': 0.25, 'max_iter': 250, 'penalty': 'l1', 'solver': 'sag'}",NaN,NaN
90,121,"{'C': 0.25, 'max_iter': 300, 'penalty': 'l1', 'solver': 'lbfgs'}",NaN,NaN
92,121,"{'C': 0.25, 'max_iter': 300, 'penalty': 'l1', 'solver': 'newton-cg'}",NaN,NaN
70,121,"{'C': 0.25, 'max_iter': 200, 'penalty': 'l1', 'solver': 'lbfgs'}",NaN,NaN


Registro del modelo

In [58]:
logit_v1 = LogisticRegression(penalty=None, C=0.2, solver="sag", max_iter=150)

logit_v1.fit(X_train_scaled, y_train) 

logit_v1_predicts = logit_v1.predict(X_test_scaled)

roc_auc = roc_auc_score(logit_v1_predicts, y_test)
print("Roc_Auc_score", roc_auc)

c:\Users\USUARIO\anaconda\Lib\site-packages\sklearn\linear_model\_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


Roc_Auc_score 0.7460861056751468


Modelo final

In [59]:
logit_modelo_final = LogisticRegression(penalty=None, C=0.2, solver="sag", max_iter=150)

X_scaled = scaler.transform(X)
logit_modelo_final.fit(X_scaled, y) 

c:\Users\USUARIO\anaconda\Lib\site-packages\sklearn\linear_model\_logistic.py:1208: UserWarning: Setting penalty=None will ignore the C and l1_ratio parameters
  warnings.warn(


LogisticRegression(C=0.2, max_iter=150, penalty=None, solver='sag')

### 4d - SVM

Entrenamiento

In [95]:
hyper_parameters = {
    "C":[0.1,1.0],
    "kernel":["linear","rbf"],
    "shrinking":[True, False],
    "probability":[True, False]
}

SVC_classifier = SVC()

start_time = datetime.now() # hora de inicio entrenamiento

# configuración de organización de hiper-parámetros
SVC_hyp_opt = GridSearchCV(estimator=SVC_classifier, param_grid=hyper_parameters, cv=10, scoring="roc_auc")

# entrenamiento de modelo
SVC_hyp_opt.fit(X_train_scaled, y_train)

end_time = datetime.now() # hora de final entrenamiento
training_duration = end_time - start_time # tiempo total entrenamiento

print("Hora inicio training: ", start_time.strftime("%Y-%m-%d %H:%M:%S"))
print("Hora final training: ", end_time.strftime("%Y-%m-%d %H:%M:%S"))
print("Duración training: ", training_duration)

Hora inicio training:  2024-06-30 21:01:27
Hora final training:  2024-06-30 21:16:53
Duración training:  0:15:26.223325


Ranking de modelo

In [96]:
pd.DataFrame(SVC_hyp_opt.cv_results_).sort_values(by="rank_test_score",ascending=True)[["rank_test_score", "params", "mean_test_score", "std_test_score"]]

,rank_test_score,params,mean_test_score,std_test_score
0,1,"{'C': 0.1, 'kernel': 'linear', 'probability': True, 'shrinking': True}",0.836518,0.018510
1,1,"{'C': 0.1, 'kernel': 'linear', 'probability': True, 'shrinking': False}",0.836518,0.018510
2,1,"{'C': 0.1, 'kernel': 'linear', 'probability': False, 'shrinking': True}",0.836518,0.018510
3,1,"{'C': 0.1, 'kernel': 'linear', 'probability': False, 'shrinking': False}",0.836518,0.018510
9,5,"{'C': 1.0, 'kernel': 'linear', 'probability': True, 'shrinking': False}",0.836204,0.018158
11,5,"{'C': 1.0, 'kernel': 'linear', 'probability': False, 'shrinking': False}",0.836204,0.018158
8,7,"{'C': 1.0, 'kernel': 'linear', 'probability': True, 'shrinking': True}",0.836200,0.018156
10,7,"{'C': 1.0, 'kernel': 'linear', 'probability': False, 'shrinking': True}",0.836200,0.018156
5,9,"{'C': 0.1, 'kernel': 'rbf', 'probability': True, 'shrinking': False}",0.819436,0.017447
7,9,"{'C': 0.1, 'kernel': 'rbf', 'probability': False, 'shrinking': False}",0.819436,0.017447


Registro del modelo

In [99]:
SVC_v1 = SVC(C=0.1, kernel="linear", shrinking=True, probability=True)

SVC_v1.fit(X_train_scaled, y_train) 

SVC_v1_predicts = SVC_v1.predict(X_test_scaled)

roc_auc = roc_auc_score(SVC_v1_predicts, y_test)
print("Roc_Auc_score", roc_auc)

Roc_Auc_score 0.7454334115512444


Modelo final

In [100]:
SVC_modelo_final =  SVC(C=0.1, kernel="linear", shrinking=True, probability=True)

X_scaled = scaler.transform(X)
SVC_modelo_final.fit(X_scaled, y) 

SVC(C=0.1, kernel='linear', probability=True)

### 4e - Árboles de decisión

Entrenamiento

In [101]:
hyper_parameters = {
    "criterion":["gini","entropy","log_loss"],
    "splitter":["best","random"],
    "max_depth":[10,20], 
    "min_samples_split":[2,4]
}

dt_classifier = DecisionTreeClassifier()

start_time = datetime.now() # hora de inicio entrenamiento

# configuración de organización de hiper-parámetros
dt_hyp_opt = GridSearchCV(estimator=dt_classifier, param_grid=hyper_parameters, cv=10, scoring="roc_auc")

# entrenamiento de modelo
dt_hyp_opt.fit(X_train_scaled, y_train)

end_time = datetime.now() # hora de final entrenamiento
training_duration = end_time - start_time # tiempo total entrenamiento

print("Hora inicio training: ", start_time.strftime("%Y-%m-%d %H:%M:%S"))
print("Hora final training: ", end_time.strftime("%Y-%m-%d %H:%M:%S"))
print("Duración training: ", training_duration)

Hora inicio training:  2024-06-30 21:19:36
Hora final training:  2024-06-30 21:19:49
Duración training:  0:00:13.734590


Ranking de modelos

In [102]:
pd.DataFrame(dt_hyp_opt.cv_results_).sort_values(by="rank_test_score",ascending=True)[["rank_test_score", "params", "mean_test_score", "std_test_score"]]

,rank_test_score,params,mean_test_score,std_test_score
19,1,"{'criterion': 'log_loss', 'max_depth': 10, 'min_samples_split': 4, 'splitter': 'random'}",0.785005,0.019216
9,2,"{'criterion': 'entropy', 'max_depth': 10, 'min_samples_split': 2, 'splitter': 'random'}",0.784931,0.017432
11,3,"{'criterion': 'entropy', 'max_depth': 10, 'min_samples_split': 4, 'splitter': 'random'}",0.784095,0.022629
17,4,"{'criterion': 'log_loss', 'max_depth': 10, 'min_samples_split': 2, 'splitter': 'random'}",0.782240,0.025455
1,5,"{'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 2, 'splitter': 'random'}",0.777618,0.017341
3,6,"{'criterion': 'gini', 'max_depth': 10, 'min_samples_split': 4, 'splitter': 'random'}",0.771652,0.025553
8,7,"{'criterion': 'entropy', 'max_depth': 10, 'min_samples_split': 2, 'splitter': 'best'}",0.754513,0.022299
18,8,"{'criterion': 'log_loss', 'max_depth': 10, 'min_samples_split': 4, 'splitter': 'best'}",0.753844,0.017622
16,9,"{'criterion': 'log_loss', 'max_depth': 10, 'min_samples_split': 2, 'splitter': 'best'}",0.751716,0.020629
10,10,"{'criterion': 'entropy', 'max_depth': 10, 'min_samples_split': 4, 'splitter': 'best'}",0.751352,0.017921


Registro del modelo

In [104]:
dt_v1 = DecisionTreeClassifier(criterion="log_loss", splitter="random", max_depth=10, min_samples_split=4)

dt_v1.fit(X_train_scaled, y_train) 

dt_v1_predicts = dt_v1.predict(X_test_scaled)

roc_auc = roc_auc_score(dt_v1_predicts, y_test)
print("Roc_Auc_score", roc_auc)

Roc_Auc_score 0.7077143302112459


Modelo final

In [106]:
dt_modelo_final = DecisionTreeClassifier(criterion="entropy", splitter="best", max_depth=10, min_samples_split=2)

X_scaled = scaler.transform(X)
dt_modelo_final.fit(X_scaled, y) 

DecisionTreeClassifier(criterion='entropy', max_depth=10)

### 4f - Random Forest

Entrenamiento

In [84]:
hyper_parameters = {
    "n_estimators":[10, 50, 100, 500, 1000],
    "criterion":["gini","entropy"],
    "max_depth":[10,20]
}

rf_classifier = RandomForestClassifier()

start_time = datetime.now() # hora de inicio entrenamiento

# configuración de organización de hiper-parámetros
rf_hyp_opt = GridSearchCV(estimator=rf_classifier, param_grid=hyper_parameters, cv=10, scoring="roc_auc")

# entrenamiento de modelo
rf_hyp_opt.fit(X_train_scaled, y_train)

end_time = datetime.now() # hora de final entrenamiento
training_duration = end_time - start_time # tiempo total entrenamiento

print("Hora inicio training: ", start_time.strftime("%Y-%m-%d %H:%M:%S"))
print("Hora final training: ", end_time.strftime("%Y-%m-%d %H:%M:%S"))
print("Duración training: ", training_duration)

Hora inicio training:  2024-06-30 20:07:36
Hora final training:  2024-06-30 20:22:37
Duración training:  0:15:00.564479


Ranking de modelos

In [85]:
pd.DataFrame(rf_hyp_opt.cv_results_).sort_values(by="rank_test_score",ascending=True)[["rank_test_score", "params", "mean_test_score", "std_test_score"]]

,rank_test_score,params,mean_test_score,std_test_score
14,1,"{'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 1000}",0.839262,0.018669
13,2,"{'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 500}",0.838958,0.019151
4,3,"{'criterion': 'gini', 'max_depth': 10, 'n_estimators': 1000}",0.837532,0.018786
2,4,"{'criterion': 'gini', 'max_depth': 10, 'n_estimators': 100}",0.837280,0.019555
3,5,"{'criterion': 'gini', 'max_depth': 10, 'n_estimators': 500}",0.837166,0.019386
12,6,"{'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 100}",0.836983,0.019734
11,7,"{'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 50}",0.836478,0.017810
1,8,"{'criterion': 'gini', 'max_depth': 10, 'n_estimators': 50}",0.835860,0.020530
10,9,"{'criterion': 'entropy', 'max_depth': 10, 'n_estimators': 10}",0.827043,0.016296
0,10,"{'criterion': 'gini', 'max_depth': 10, 'n_estimators': 10}",0.825138,0.022044


Registro del Modelo

In [39]:
rf_v1 = RandomForestClassifier(criterion="entropy", max_depth=10, n_estimators=500)

rf_v1.fit(X_train_scaled, y_train) 

rf_v1_predicts = rf_v1.predict(X_test_scaled)

roc_auc = roc_auc_score(rf_v1_predicts, y_test)
print("Roc_Auc_score", roc_auc)

Roc_Auc_score 0.7395054690372044


Modelo Final

In [46]:
rf_modelo_final = RandomForestClassifier(criterion="entropy", max_depth=10, n_estimators=500)

X_scaled = scaler.transform(X)
rf_modelo_final.fit(X_scaled, y) 

RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=500)

### 4h - Análisis de Discriminante Cuadrático

Entrenamiento

In [107]:
hyper_parameters = {
    "priors":[None, [0.25, 0.75], [0.5, 0.5], [0.75, 0.25]],
    "reg_param":[0.0, 0.5],
    "store_covariance":[True, False],
    "tol":[1.0e-4, 1.0e-2]
}

QDA = QuadraticDiscriminantAnalysis()

start_time = datetime.now() # hora de inicio entrenamiento

# configuración de organización de hiper-parámetros
QDA_hyp_opt = GridSearchCV(estimator=QDA, param_grid=hyper_parameters, cv=10, scoring="roc_auc")

# entrenamiento de modelo
QDA_hyp_opt.fit(X_train_scaled, y_train)

end_time = datetime.now() # hora de final entrenamiento
training_duration = end_time - start_time # tiempo total entrenamiento

print("Hora inicio training: ", start_time.strftime("%Y-%m-%d %H:%M:%S"))
print("Hora final training: ", end_time.strftime("%Y-%m-%d %H:%M:%S"))
print("Duración training: ", training_duration)

c:\Users\USUARIO\anaconda\Lib\site-packages\sklearn\discriminant_analysis.py:949: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\USUARIO\anaconda\Lib\site-packages\sklearn\discriminant_analysis.py:949: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\USUARIO\anaconda\Lib\site-packages\sklearn\discriminant_analysis.py:949: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\USUARIO\anaconda\Lib\site-packages\sklearn\discriminant_analysis.py:949: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\USUARIO\anaconda\Lib\site-packages\sklearn\discriminant_analysis.py:949: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\USUARIO\anaconda\Lib\site-packages\sklearn\discriminant_analysis.py:949: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\USUARIO\anaconda\Li

Hora inicio training:  2024-06-30 21:25:19
Hora final training:  2024-06-30 21:25:25
Duración training:  0:00:06.222387


c:\Users\USUARIO\anaconda\Lib\site-packages\sklearn\discriminant_analysis.py:949: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\USUARIO\anaconda\Lib\site-packages\sklearn\discriminant_analysis.py:949: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\USUARIO\anaconda\Lib\site-packages\sklearn\discriminant_analysis.py:949: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\USUARIO\anaconda\Lib\site-packages\sklearn\discriminant_analysis.py:949: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\USUARIO\anaconda\Lib\site-packages\sklearn\discriminant_analysis.py:949: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\USUARIO\anaconda\Lib\site-packages\sklearn\discriminant_analysis.py:949: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
c:\Users\USUARIO\anaconda\Li

Ranking modelos

In [108]:
pd.DataFrame(QDA_hyp_opt.cv_results_).sort_values(by="rank_test_score",ascending=True)[["rank_test_score", "params", "mean_test_score", "std_test_score"]]

,rank_test_score,params,mean_test_score,std_test_score
15,1,"{'priors': [0.25, 0.75], 'reg_param': 0.5, 'store_covariance': False, 'tol': 0.01}",0.825006,0.017784
29,1,"{'priors': [0.75, 0.25], 'reg_param': 0.5, 'store_covariance': True, 'tol': 0.01}",0.825006,0.017784
28,1,"{'priors': [0.75, 0.25], 'reg_param': 0.5, 'store_covariance': True, 'tol': 0.0001}",0.825006,0.017784
23,1,"{'priors': [0.5, 0.5], 'reg_param': 0.5, 'store_covariance': False, 'tol': 0.01}",0.825006,0.017784
22,1,"{'priors': [0.5, 0.5], 'reg_param': 0.5, 'store_covariance': False, 'tol': 0.0001}",0.825006,0.017784
21,1,"{'priors': [0.5, 0.5], 'reg_param': 0.5, 'store_covariance': True, 'tol': 0.01}",0.825006,0.017784
20,1,"{'priors': [0.5, 0.5], 'reg_param': 0.5, 'store_covariance': True, 'tol': 0.0001}",0.825006,0.017784
30,1,"{'priors': [0.75, 0.25], 'reg_param': 0.5, 'store_covariance': False, 'tol': 0.0001}",0.825006,0.017784
14,1,"{'priors': [0.25, 0.75], 'reg_param': 0.5, 'store_covariance': False, 'tol': 0.0001}",0.825006,0.017784
13,1,"{'priors': [0.25, 0.75], 'reg_param': 0.5, 'store_covariance': True, 'tol': 0.01}",0.825006,0.017784


Registro del modelo

In [109]:
QDA_v1 = QuadraticDiscriminantAnalysis(priors=[0.25, 0.75], reg_param=0.5, store_covariance=False, tol=0.01)

QDA_v1.fit(X_train_scaled, y_train) 

QDA_v1_predicts = QDA_v1.predict(X_test_scaled)

roc_auc = roc_auc_score(QDA_v1_predicts, y_test)
print("Roc_Auc_score", roc_auc)

Roc_Auc_score 0.6362754473203281


c:\Users\USUARIO\anaconda\Lib\site-packages\sklearn\discriminant_analysis.py:949: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


Modelo final

In [110]:
QDA_modelo_final = QuadraticDiscriminantAnalysis(priors=[0.25, 0.75], reg_param=0.5, store_covariance=False, tol=0.01)

X_scaled = scaler.transform(X)
QDA_modelo_final.fit(X_scaled, y) 

c:\Users\USUARIO\anaconda\Lib\site-packages\sklearn\discriminant_analysis.py:949: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


QuadraticDiscriminantAnalysis(priors=[0.25, 0.75], reg_param=0.5, tol=0.01)

### 4i AdaBoost

Entrenamiento

In [111]:
hyper_parameters = {
    "n_estimators":[10, 50, 75],
    "learning_rate":[0.5, 1.0]
}

ADA = AdaBoostClassifier()

start_time = datetime.now() # hora de inicio entrenamiento

# configuración de organización de hiper-parámetros
ADA_hyp_opt = GridSearchCV(estimator=ADA, param_grid=hyper_parameters, cv=10, scoring="roc_auc")

# entrenamiento de modelo
ADA_hyp_opt.fit(X_train_scaled, y_train)

end_time = datetime.now() # hora de final entrenamiento
training_duration = end_time - start_time # tiempo total entrenamiento

print("Hora inicio training: ", start_time.strftime("%Y-%m-%d %H:%M:%S"))
print("Hora final training: ", end_time.strftime("%Y-%m-%d %H:%M:%S"))
print("Duración training: ", training_duration)

c:\Users\USUARIO\anaconda\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\USUARIO\anaconda\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\USUARIO\anaconda\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users\USUARIO\anaconda\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(
c:\Users

Hora inicio training:  2024-06-30 21:28:22
Hora final training:  2024-06-30 21:29:03
Duración training:  0:00:40.129489


Ranking de modelos

In [112]:
pd.DataFrame(ADA_hyp_opt.cv_results_).sort_values(by="rank_test_score",ascending=True)[["rank_test_score", "params", "mean_test_score", "std_test_score"]]

,rank_test_score,params,mean_test_score,std_test_score
1,1,"{'learning_rate': 0.5, 'n_estimators': 50}",0.841863,0.017067
2,2,"{'learning_rate': 0.5, 'n_estimators': 75}",0.841363,0.017959
4,3,"{'learning_rate': 1.0, 'n_estimators': 50}",0.841321,0.017850
5,4,"{'learning_rate': 1.0, 'n_estimators': 75}",0.840605,0.017631
3,5,"{'learning_rate': 1.0, 'n_estimators': 10}",0.836148,0.016393
0,6,"{'learning_rate': 0.5, 'n_estimators': 10}",0.834680,0.015952


Registro del modelo

In [115]:
ADA_v1 = AdaBoostClassifier(n_estimators=50, learning_rate=0.5)

ADA_v1.fit(X_train_scaled, y_train) 

ADA_v1_predicts = ADA_v1.predict(X_test_scaled)

roc_auc = roc_auc_score(ADA_v1_predicts, y_test)
print("Roc_Auc_score", roc_auc)

c:\Users\USUARIO\anaconda\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


Roc_Auc_score 0.7384586687610446


Modelo final

In [116]:
ADA_modelo_final = AdaBoostClassifier(n_estimators=50, learning_rate=0.5)

X_scaled = scaler.transform(X)
ADA_modelo_final.fit(X_scaled, y)

c:\Users\USUARIO\anaconda\Lib\site-packages\sklearn\ensemble\_weight_boosting.py:527: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoostClassifier(learning_rate=0.5)

### 4j Gradient Boosting

Entrenamiento

In [117]:
hyper_parameters = {
    "loss":["log_loss","exponential"],
    "learning_rate":[0.1, 0.2],
    "n_estimators":[50, 100, 150]
}

GB = GradientBoostingClassifier()

start_time = datetime.now() # hora de inicio entrenamiento

# configuración de organización de hiper-parámetros
GB_hyp_opt = GridSearchCV(estimator=GB, param_grid=hyper_parameters, cv=10, scoring="roc_auc")

# entrenamiento de modelo
GB_hyp_opt.fit(X_train_scaled, y_train)

end_time = datetime.now() # hora de final entrenamiento
training_duration = end_time - start_time # tiempo total entrenamiento

print("Hora inicio training: ", start_time.strftime("%Y-%m-%d %H:%M:%S"))
print("Hora final training: ", end_time.strftime("%Y-%m-%d %H:%M:%S"))
print("Duración training: ", training_duration)

Hora inicio training:  2024-06-30 21:30:57
Hora final training:  2024-06-30 21:33:56
Duración training:  0:02:59.213503


Ranking de modelos

In [118]:
pd.DataFrame(GB_hyp_opt.cv_results_).sort_values(by="rank_test_score",ascending=True)[["rank_test_score", "params", "mean_test_score", "std_test_score"]]

,rank_test_score,params,mean_test_score,std_test_score
0,1,"{'learning_rate': 0.1, 'loss': 'log_loss', 'n_estimators': 50}",0.843730,0.017365
3,2,"{'learning_rate': 0.1, 'loss': 'exponential', 'n_estimators': 50}",0.843487,0.017409
4,3,"{'learning_rate': 0.1, 'loss': 'exponential', 'n_estimators': 100}",0.841812,0.018066
6,4,"{'learning_rate': 0.2, 'loss': 'log_loss', 'n_estimators': 50}",0.841674,0.018915
1,5,"{'learning_rate': 0.1, 'loss': 'log_loss', 'n_estimators': 100}",0.841382,0.018881
9,6,"{'learning_rate': 0.2, 'loss': 'exponential', 'n_estimators': 50}",0.840898,0.017592
5,7,"{'learning_rate': 0.1, 'loss': 'exponential', 'n_estimators': 150}",0.839641,0.019112
2,8,"{'learning_rate': 0.1, 'loss': 'log_loss', 'n_estimators': 150}",0.838761,0.019335
10,9,"{'learning_rate': 0.2, 'loss': 'exponential', 'n_estimators': 100}",0.836211,0.020553
7,10,"{'learning_rate': 0.2, 'loss': 'log_loss', 'n_estimators': 100}",0.835567,0.019026


Registro del modelo

In [119]:
GB_v1 = GradientBoostingClassifier(loss="log_loss", learning_rate=0.1, n_estimators=50)

GB_v1.fit(X_train_scaled, y_train) 

GB_v1_predicts = GB_v1.predict(X_test_scaled)

roc_auc = roc_auc_score(GB_v1_predicts, y_test)
print("Roc_Auc_score", roc_auc)

Roc_Auc_score 0.7435342956959498


Modelo final

In [ ]:
GB_modelo_final = GradientBoostingClassifier(loss="log_loss", learning_rate=0.1, n_estimators=100)

X_scaled = scaler.transform(X)
GB_modelo_final.fit(X_scaled, y)

### 4k XGBoost

Entrenamiento

In [122]:
hyper_parameters = {
    "booster":["gblinear","dart"],
    "eta":[0.3, 0.75],
    "gamma":[0, 0.5, 5],
    "max_depth":[3, 9]
}

XGB = XGBClassifier()

start_time = datetime.now() # hora de inicio entrenamiento

# configuración de organización de hiper-parámetros
XGB_hyp_opt = GridSearchCV(estimator=XGB, param_grid=hyper_parameters, cv=10, scoring="roc_auc")

# entrenamiento de modelo
XGB_hyp_opt.fit(X_train_scaled, y_train)

end_time = datetime.now() # hora de final entrenamiento
training_duration = end_time - start_time # tiempo total entrenamiento

print("Hora inicio training: ", start_time.strftime("%Y-%m-%d %H:%M:%S"))
print("Hora final training: ", end_time.strftime("%Y-%m-%d %H:%M:%S"))
print("Duración training: ", training_duration)

c:\Users\USUARIO\anaconda\Lib\site-packages\xgboost\core.py:158: UserWarning: [21:54:33] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "gamma", "max_depth" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\USUARIO\anaconda\Lib\site-packages\xgboost\core.py:158: UserWarning: [21:54:34] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "gamma", "max_depth" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\USUARIO\anaconda\Lib\site-packages\xgboost\core.py:158: UserWarning: [21:54:34] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-06abd128ca6c1688d-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "gamma", "max_depth" } are not used.

  warnings.warn(smsg, UserWarning)
c:\Users\USUARIO\anaconda\Lib\site-pa

Hora inicio training:  2024-06-30 21:54:33
Hora final training:  2024-06-30 22:10:58
Duración training:  0:16:24.829758


Ranking de modelos

In [123]:
pd.DataFrame(XGB_hyp_opt.cv_results_).sort_values(by="rank_test_score",ascending=True)[["rank_test_score", "params", "mean_test_score", "std_test_score"]]

,rank_test_score,params,mean_test_score,std_test_score
16,1,"{'booster': 'dart', 'eta': 0.3, 'gamma': 5, 'max_depth': 3}",0.842768,0.017433
11,2,"{'booster': 'gblinear', 'eta': 0.75, 'gamma': 5, 'max_depth': 9}",0.842340,0.017306
8,3,"{'booster': 'gblinear', 'eta': 0.75, 'gamma': 0.5, 'max_depth': 3}",0.842325,0.017372
7,4,"{'booster': 'gblinear', 'eta': 0.75, 'gamma': 0, 'max_depth': 9}",0.842237,0.017321
9,5,"{'booster': 'gblinear', 'eta': 0.75, 'gamma': 0.5, 'max_depth': 9}",0.842204,0.017313
6,6,"{'booster': 'gblinear', 'eta': 0.75, 'gamma': 0, 'max_depth': 3}",0.842151,0.017404
10,7,"{'booster': 'gblinear', 'eta': 0.75, 'gamma': 5, 'max_depth': 3}",0.842125,0.017349
14,8,"{'booster': 'dart', 'eta': 0.3, 'gamma': 0.5, 'max_depth': 3}",0.841700,0.018224
0,9,"{'booster': 'gblinear', 'eta': 0.3, 'gamma': 0, 'max_depth': 3}",0.841699,0.017581
2,10,"{'booster': 'gblinear', 'eta': 0.3, 'gamma': 0.5, 'max_depth': 3}",0.841686,0.017634


Registro del modelo

In [124]:
XGB_v1 = XGBClassifier(booster="dart", eta=0.3, gamma=5, max_depth=3)

XGB_v1.fit(X_train_scaled, y_train) 

XGB_v1_predicts = XGB_v1.predict(X_test_scaled)

roc_auc = roc_auc_score(XGB_v1_predicts, y_test)
print("Roc_Auc_score", roc_auc)

Roc_Auc_score 0.7455430917232


Modelo final

In [126]:
XGB_modelo_final = XGBClassifier(booster="dart", eta=0.3, gamma=5, max_depth=3)

X_scaled = scaler.transform(X)
XGB_modelo_final.fit(X_scaled, y)

XGBClassifier(base_score=None, booster='dart', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eta=0.3, eval_metric=None,
              feature_types=None, gamma=5, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

### 4l LGBM

Entrenamiento

In [127]:
hyper_parameters = {
    "boosting":["gbdt","rf","dart"],
    "data_sample_strategy":["bagging", "goss"],
    "num_iterations":[50, 100, 200]
}

LGBM = LGBMClassifier()

start_time = datetime.now() # hora de inicio entrenamiento

# configuración de organización de hiper-parámetros
LGBM_hyp_opt = GridSearchCV(estimator=LGBM, param_grid=hyper_parameters, cv=10, scoring="roc_auc")

# entrenamiento de modelo
LGBM_hyp_opt.fit(X_train_scaled, y_train)

end_time = datetime.now() # hora de final entrenamiento
training_duration = end_time - start_time # tiempo total entrenamiento

print("Hora inicio training: ", start_time.strftime("%Y-%m-%d %H:%M:%S"))
print("Hora final training: ", end_time.strftime("%Y-%m-%d %H:%M:%S"))
print("Duración training: ", training_duration)

c:\Users\USUARIO\anaconda\Lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
c:\Users\USUARIO\anaconda\Lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Info] Number of positive: 1204, number of negative: 3233
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002464 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 603
[LightGBM] [Info] Number of data points in the train set: 4437, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.271355 -> initscore=-0.987761
[LightGBM] [Info] Start training from score -0.987761
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Wa

c:\Users\USUARIO\anaconda\Lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt
[LightGBM] [Info] Number of positive: 1203, number of negative: 3234
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001333 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 603
[LightGBM] [Info] Number of data points in the train set: 4437, number of used features: 11
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.271129 -> initscore=-0.988901
[LightGBM] [Info] Start training from score -0.988901
[LightGBM] [Warning] boosting is set=gbdt, boosting_type=gbdt will be ignored. Current value: boosting=gbdt


c:\Users\USUARIO\anaconda\Lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
c:\Users\USUARIO\anaconda\Lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


...

[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Info] Number of positive: 1203, number of negative: 3234
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000651 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 603
[LightGBM] [Info] Number of data points in the train set: 4437, number of used features: 11
[LightGBM] [Info] Using GOSS
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.271129 -> initscore=-0.988901
[LightGBM] [Info] Start training from score -0.988901


c:\Users\USUARIO\anaconda\Lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] boosting is set=rf, boosting_type=gbdt will be ignored. Current value: boosting=rf
[LightGBM] [Warning] boosting is set=rf, boosting_type=gbdt will be ignored. Current value: boosting=rf
[LightGBM] [Info] Number of positive: 1337, number of negative: 3593
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000527 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 603
[LightGBM] [Info] Number of data points in the train set: 4930, number of used features: 11
[LightGBM] [Info] Using GOSS
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.271197 -> initscore=-0.988559
[LightGBM] [Info] Start training from score -0.988559


c:\Users\USUARIO\anaconda\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
30 fits failed out of a total of 180.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
30 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\USUARIO\anaconda\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\USUARIO\anaconda\Lib\site-packages\lightgbm\sklearn.py", line 1268, in fit
    super().fit(
  File "c:\Users\USUARIO\anaconda\Lib\site-packages\lightgbm\sklearn.py", line 954, in fit
    self._Booster = train(
                    ^^^^^^
  File "c:\Users\USUARIO\anaconda\Lib\site-packages\l

Hora inicio training:  2024-06-30 22:16:08
Hora final training:  2024-06-30 22:20:08
Duración training:  0:04:00.163966


Ranking de modelos

In [128]:
pd.DataFrame(LGBM_hyp_opt.cv_results_).sort_values(by="rank_test_score",ascending=True)[["rank_test_score", "params", "mean_test_score", "std_test_score"]]

,rank_test_score,params,mean_test_score,std_test_score
11,1,"{'boosting': 'rf', 'data_sample_strategy': 'goss', 'num_iterations': 200}",0.841187,0.016932
10,2,"{'boosting': 'rf', 'data_sample_strategy': 'goss', 'num_iterations': 100}",0.840555,0.016929
9,3,"{'boosting': 'rf', 'data_sample_strategy': 'goss', 'num_iterations': 50}",0.840043,0.017697
12,4,"{'boosting': 'dart', 'data_sample_strategy': 'bagging', 'num_iterations': 50}",0.838335,0.020766
13,5,"{'boosting': 'dart', 'data_sample_strategy': 'bagging', 'num_iterations': 100}",0.836676,0.020052
15,6,"{'boosting': 'dart', 'data_sample_strategy': 'goss', 'num_iterations': 50}",0.836454,0.022328
0,7,"{'boosting': 'gbdt', 'data_sample_strategy': 'bagging', 'num_iterations': 50}",0.835673,0.022566
16,8,"{'boosting': 'dart', 'data_sample_strategy': 'goss', 'num_iterations': 100}",0.834309,0.021698
14,9,"{'boosting': 'dart', 'data_sample_strategy': 'bagging', 'num_iterations': 200}",0.833515,0.019930
3,10,"{'boosting': 'gbdt', 'data_sample_strategy': 'goss', 'num_iterations': 50}",0.830327,0.023801


Registro del modelo

In [129]:
LGBM_v1 = LGBMClassifier(boosting="rf", data_sample_strategy="goss", num_iterations=200)

LGBM_v1.fit(X_train_scaled, y_train) 

LGBM_v1_predicts = LGBM_v1.predict(X_test_scaled)

roc_auc = roc_auc_score(LGBM_v1_predicts, y_test)
print("Roc_Auc_score", roc_auc)

c:\Users\USUARIO\anaconda\Lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] boosting is set=rf, boosting_type=gbdt will be ignored. Current value: boosting=rf
[LightGBM] [Warning] boosting is set=rf, boosting_type=gbdt will be ignored. Current value: boosting=rf
[LightGBM] [Info] Number of positive: 1337, number of negative: 3593
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001065 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 603
[LightGBM] [Info] Number of data points in the train set: 4930, number of used features: 11
[LightGBM] [Info] Using GOSS
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.271197 -> initscore=-0.988559
[LightGBM] [Info] Start training from score -0.988559
[LightGBM] [Warning] boosting is set=rf, boosting_type=gbdt will be ignored. Current value: boosting=rf
Roc_Auc_score 0.7435969058874087


Modelo final

In [130]:
LGBM_modelo_final = LGBMClassifier(boosting="rf", data_sample_strategy="goss", num_iterations=200)

X_scaled = scaler.transform(X)
LGBM_modelo_final.fit(X_scaled, y)

c:\Users\USUARIO\anaconda\Lib\site-packages\lightgbm\engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] boosting is set=rf, boosting_type=gbdt will be ignored. Current value: boosting=rf
[LightGBM] [Warning] boosting is set=rf, boosting_type=gbdt will be ignored. Current value: boosting=rf
[LightGBM] [Info] Number of positive: 1869, number of negative: 5174
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.156124 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 603
[LightGBM] [Info] Number of data points in the train set: 7043, number of used features: 11
[LightGBM] [Info] Using GOSS
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.265370 -> initscore=-1.018243
[LightGBM] [Info] Start training from score -1.018243


LGBMClassifier(boosting='rf', data_sample_strategy='goss', num_iterations=200)